# Introduction

In the previous notebook, we used perlin noise to randomly generate input data to run GOSPL simulations with. We then ran hundred/thousands of GOSPL trials and created a training data set that can be used to train various neural network (NN) models. In this notebook, we will train NN models to replicate GOSPL simulation.

# Imports and Usefull Functions

In [ ]:
import os
import time
import numpy as np
import pyvista as pv
from PIL import Image
import tensorflow as tf
import keras.backend as K
from keras.models import Model
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from keras.initializers import RandomNormal

# Load feature and target data
def loadDataset(dataSetDir='./Data/TrainingDataSets/gosplData.npz'):
    dataSet = np.load(dataSetDir)
    features = dataSet['features']
    targets = dataSet['targets']
    return features, targets

#Coordinate transformation from cartesian to polar
def cartesianToPolarCoords(XYZ, useLonLat=True):
    X, Y, Z = XYZ[:, 0], XYZ[:, 1], XYZ[:, 2]
    R = (X**2 + Y**2 + Z**2)**0.5
    theta = np.arctan2(Y, X)
    phi = np.arccos(Z / R)

    # Return results either in spherical polar or leave it in radians
    if useLonLat == True:
        theta, phi = np.degrees(theta), np.degrees(phi)
        lon, lat = theta - 180, 90 - phi
        lon[lon < -180] = lon[lon < -180] + 360
        return R, lon, lat
    else:
        return R, theta, phi

#Coordinate transformation from spherical polar to cartesian
def polarToCartesian(radius, theta, phi, useLonLat=True):
    if useLonLat == True:
        theta, phi = np.radians(theta+180.), np.radians(90. - phi)
    X = radius * np.cos(theta) * np.sin(phi)
    Y = radius * np.sin(theta) * np.sin(phi)
    Z = radius * np.cos(phi)

    # Return data either as a list of XYZ coordinates or as a single XYZ coordinate
    if (type(X) == np.ndarray):
        return np.stack((X, Y, Z), axis=1)
    else:
        return np.array([X, Y, Z])

# Visualizing the Training Data

Before we continue, we visualize the data to make sure that they are in the correct format and haven't been corrupted along the way. These functions will also be usefull to visualize the output of the tensorflow models later on.

In the code bellow, we provide a visualization of the data in the image format. This format is more representative of how tensorflow will actually see the data.

In [ ]:
# Display an image representation of a sample image
def visualizeAsImages(featureImage, targetImage):
    featureImage -= np.min(featureImage, axis=(0, 1))
    featureImage /= np.max(featureImage, axis=(0, 1))
    targetImage -= np.min(targetImage, axis=(0, 1))
    targetImage /= np.max(targetImage, axis=(0, 1))
    #targetImage[:, :, 2] **= 0.125

    #Set up plotting figure
    fig, axis = plt.subplots(1, 3)
    fig.set_figheight(12)
    fig.set_figwidth(18)
    axis[0].imshow(featureImage[:, :, 0])
    axis[1].imshow(featureImage[:, :, 1])
    axis[2].imshow(targetImage)
    axis[0].set_title('Initial Elevations')
    axis[1].set_title('Tectonic Uplift')
    axis[2].set_title('Combined Output Data')
    axis[0].set_xticks([])
    axis[0].set_yticks([])
    axis[1].set_xticks([])
    axis[1].set_yticks([])
    axis[2].set_xticks([])
    axis[2].set_yticks([])
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    return fig, axis

# Load dataset and visualize a sample from it
features, targets = loadDataset('./Data/TrainingDataSets/SmallExampleDataset.npz')
fig, axis = visualizeAsImages(features[0], targets[0])

Although the data used will be in an image format, it is meant to represent a spherical planet. The code bellow provides a spherical representation of what the data looks like, with an exagerated terrain based on the final elevations. This function will also be compatible with the outputs of the tensorflow models.

In the top left corner of the ITK plotter window, there is a dropdown meny represented by 3 lines. Within this dropdown menu, we can chose which parameter to color code the mesh with.

In [ ]:
# Create pyvista mesh object with exagerated terrains and data attached to it
def createExageratedMesh(featureSample, targetSample, amplificationFactor=90, earthRadius=6378137):
    
    # Reshape sample data and re-insert placeholder values (zeros) for north and south poles
    imageDims = np.array(features.shape[1:3])
    featureSample = featureSample.reshape(imageDims[0] * imageDims[1], 2)
    featureSample = np.insert(featureSample, 0, 0, axis=0)
    featureSample = np.insert(featureSample, 0, 0, axis=0)
    targetSample = targetSample.reshape(imageDims[0] * imageDims[1], 3)
    targetSample = np.insert(targetSample, 0, 0, axis=0)
    targetSample = np.insert(targetSample, 0, 0, axis=0)

    # Create a planet mesh with exagerated final elevations
    uvSphere = pv.Sphere(theta_resolution=imageDims[0], phi_resolution=imageDims[1]+2)
    r, lon, lat = cartesianToPolarCoords(uvSphere.points)
    exegeratedRadius = earthRadius + amplificationFactor * targetSample[:, 0]
    exageratedXYZ = polarToCartesian(exegeratedRadius, lon, lat)
    exageratedMesh = pv.PolyData(exageratedXYZ.T, uvSphere.faces)

    # Attach data to the exagerated mesh
    exageratedMesh['Initial Elevations'] = featureSample[:, 0]
    exageratedMesh['Tectonic Uplift'] = featureSample[:, 1]
    exageratedMesh['Final Elevations'] = targetSample[:, 0]
    exageratedMesh['Erosion Deposition'] = targetSample[:, 1]
    exageratedMesh['Flow Accumilation'] = targetSample[:, 2]**0.125
    return exageratedMesh

# Create exagerated mesh
sampleNumber = 10
exageratedMesh = createExageratedMesh(features[sampleNumber], targets[sampleNumber])

# Plot the results
plotter = pv.PlotterITK()
plotter.add_mesh(exageratedMesh, scalars='Final Elevations')
plotter.show()

# Sigmoids and Logit Functions

Machine learning models generally don't perform well when the training data is within some random range, instead we want all data to be within a range of $[-1, 1]$, or sometimes within $[0, 1]$. We also don't want to use the standard normalization technique, because this would artificially introduce a lower and upper bound on our data. We can use the sigmoid function to bring the data into the range of $[0, 1]$, and a logit function to bring it back to the original data domain.

The usual sigmoid function and logit is given bellow. Here $\alpha$ is the gain, and corresponds to the steepness of the sigmoid function.

$$\text{sigmoid}(x, \alpha, \mu) = \frac{1}{1 + e^{-\alpha (x - \mu)}}$$

$$\text{logit}(x, \alpha, \mu) = \mu + \log \Big( {\frac{x}{1 - x}} \Big) \Big/ \alpha$$

We want to chose a gain $\alpha$ and centre $\mu$ such that most of the variance in our data falls within the changing region of the sigmoid function. Therefore, we calculate a gain and centre based on a specified range in which the data varies by about 2 standard deviations.

In [ ]:
# Used to bring data to 0-1
def sigmoid(x, minMax=np.array([-1, 1])):
    centre = (minMax[0] + minMax[1]) / 2
    gain = 8 / (minMax[1] - minMax[0])
    return 1 / (1 + np.exp(- gain * (x - centre)))

# Inverse function to bring data back to original domain
def logit(x, minMax=np.array([-1, 1])):
    centre = (minMax[0] + minMax[1]) / 2
    gain = 8 / (minMax[1] - minMax[0])
    return centre + np.log(x / (1 - x)) / gain

# Create XY data to demonstrate functions with
minMax = np.array([-4, 4])
x = np.arange(-8, 8, 0.01)
y = sigmoid(x, minMax=minMax)
a = np.arange(0.01, 0.99, 0.01)
b = logit(a, minMax=minMax)

# Plot functions
fig, axis = plt.subplots(1, 2)
fig.set_figheight(4)
fig.set_figwidth(16)
axis[0].plot(x, y)
axis[1].plot(a, b)
axis[0].set_title('Sigmoid')
axis[1].set_title('Logit')
axis[0].grid()
axis[1].grid()
plt.subplots_adjust(wspace=0.1, hspace=0.1)
plt.show()

When using the above functions, we want to choose an appropriate range for each variable to avoid all values being too close to 0 or 1, but rather something inbetween. Otherwise the neural network may not *see* the relevant details of our data set.

We can use the figures bellow to figure out what range our data typically lies within, and decide what gain is suitable for each attribute.

In [ ]:
features, targets = loadDataset(dataSetDir='./Data/TrainingDataSets/SmallerDataSet10Examples.npz')
feats = features[0:5].reshape(5 * 512 * 256, 2)
targs = targets[0:5].reshape(5 * 512 * 256, 3)

fig, axis = plt.subplots(5, 1)
fig.set_figheight(12)
fig.set_figwidth(16)
axis[0].plot(feats[:, 0], linewidth=0.05)
axis[1].plot(feats[:, 1], linewidth=0.05)
axis[2].plot(targs[:, 0], linewidth=0.05)
axis[3].plot(targs[:, 1], linewidth=0.05)
axis[4].plot(targs[:, 2]**0.25, linewidth=0.05)
axis[0].set_title('Min: {}, Max: {}'.format(np.min(feats[:, 0]), np.max(feats[:, 0])))
axis[1].set_title('Min: {}, Max: {}'.format(np.min(feats[:, 1]), np.max(feats[:, 1])))
axis[2].set_title('Min: {}, Max: {}'.format(np.min(targs[:, 0]), np.max(targs[:, 0])))
axis[3].set_title('Min: {}, Max: {}'.format(np.min(targs[:, 1]), np.max(targs[:, 1])))
axis[4].set_title('Min: {}, Max: {}'.format(np.min(targs[:, 2]**0.125), np.max(targs[:, 2]**0.125)))
for i in range(5):
    axis[i].set_xticks([])

In [ ]:
# Brings data to range of 0-1 without imposing a strict upper bounds
def passDataThroughSigmoid(features, targets, 
                           minMaxes = np.array([[-4000, 1000],
                                                [0, 0.0002],
                                                [0, 12000],
                                                [-1000, 3000],
                                                [0, 1000]])):
    newFeatures, newTargets = [], []
    newFeatures.append(sigmoid(features.T[0], minMax=minMaxes[0]))
    newFeatures.append(sigmoid(features.T[1], minMax=minMaxes[1]))
    newTargets.append(sigmoid(targets.T[0], minMax=minMaxes[2]))
    newTargets.append(sigmoid(targets.T[1], minMax=minMaxes[3]))
    newTargets.append(sigmoid(targets.T[2]**0.25, minMax=minMaxes[4]))
    return np.array(newFeatures).T, np.array(newTargets).T

# Brings the output of the model back into the original domain
def passDataThroughLogit(features, targets, 
                           minMaxes = np.array([[-4000, 1000],
                                                [0, 0.0002],
                                                [0, 12000],
                                                [-1000, 3000],
                                                [0, 1000]])):
    newFeatures, newTargets = [], []
    newFeatures.append(logit(features.T[0], minMax=minMaxes[0]))
    newFeatures.append(logit(features.T[1], minMax=minMaxes[1]))
    newTargets.append(logit(targets.T[0], minMax=minMaxes[2]))
    newTargets.append(logit(targets.T[1], minMax=minMaxes[3]))
    newTargets.append(logit(targets.T[2], minMax=minMaxes[4])**4)
    return np.array(newFeatures).T, np.array(newTargets).T

# Specify min max ranges for sigmoid and logit functions for all parameters
minMaxes = np.array([
    [-4000, 1000],
    [0, 0.0002],
    [0, 12000],
    [-1000, 3000],
    [0, 1000]])

#features, targets = loadDataset()
newFeatures, newTargets = passDataThroughSigmoid(features, targets, minMaxes)

plt.figure(figsize=(12, 12))
plt.imshow(newTargets[0])
plt.show()

In the future, it would be desirable to have this sigmoid and logit transformation take place within the tensorflow model itself such that the gain and centre are trainable parameters.

# The Generator - Unet Model

In the context of a GAN, the generator will attempt will take initial topography and tectonic uplift as inputs, and attempt to generate a *fake* output that fools the discriminateor into thinking it came from the *real* data set, which in our case has been generated by GOSPL. In previous notebooks, we found the Unet model seems to be a promising neural network architecture for our generator. 

The Unet model is a NN model in which almost all layers are convolutional. The image bellow ([taken from here](https://upload.wikimedia.org/wikipedia/commons/2/2b/Example_architecture_of_U-Net_for_producing_k_256-by-256_image_masks_for_a_256-by-256_RGB_image.png)) is an example diagram of how a Unet model can be structured. It is composed of a series of downsampling (aka encoding) layers, in which convolution and pooling layers reduce the data in it's image dimension, but the number of convolutional filters increase in each layer. Noise is then introduced in the botom most layer, as it prevents overfitting of the data (Why noise helps is still an open research question).

A series of upsampling convolutional layers are used to bring the data back into it's desired image dimensions. The output of each upsampling layer is merged with the corresponding downsampling layer of the same depth, which allows data to bypass the deeper encoding/decoding layers.

<br>
<div>
<img src="../NotebookImages/ExampleUnet.png" width="600">
</div>



The code bellow is taken and modified from [*nanoxas/sketch-to-terrain* Github](https://github.com/nanoxas/sketch-to-terrain/blob/master/model.py), where [Eric Guerin et. al.](https://hal.archives-ouvertes.fr/hal-01583706/file/tog.pdf) had a similar goal in mind.

In [ ]:
#Here we define the structure of the generator Unet model
def getUnet(imageDims):
    imageWidth, imageHeight = imageDims[1], imageDims[0]
    
    #Input layer
    inputs = layers.Input((imageHeight, imageWidth, 2))
    
    #A few convolutional layers with maxpooling
    #This is the encoder part of the model that interprets the input image
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)
    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    
    #Their last convolution layer in their encoder seems to have skip layer with noise introduced to it
    #Noise is often used to avoid the overfiting of a machine learning model
    noise = layers.Input((K.int_shape(conv5)[1], K.int_shape(conv5)[2], K.int_shape(conv5)[3]))
    conv5 = layers.Concatenate()([conv5, noise])
    
    #From my understanding, upsampling is used to increase the weight of data in the minority class
    #Skip layer (connects conv4 layer directly to up6 layer), and more convolutional layers
    up6 = layers.Conv2D(512, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv5))
    merge6 = layers.Concatenate()([conv4, up6])
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv6)
    
    up7 = layers.Conv2D(256, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv6))
    merge7 = layers.Concatenate()([conv3, up7])
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv7)
    
    up8 = layers.Conv2D(128, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv7))
    merge8 = layers.Concatenate()([conv2, up8])
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv8)
    
    up9 = layers.Conv2D(64, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv8))
    merge9 = layers.Concatenate()([conv1, up9])
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv9)
    conv9 = layers.Conv2D(32, 3, activation='relu', padding='same')(conv9)
    conv10 = layers.Conv2D(3, 1, activation='tanh')(conv9)
    
    #Create and return the final model
    model = Model(inputs=[inputs, noise], outputs=conv10)
    return model

# Create a generator model and print a summary of it's layers
#features, targets = loadDataset()
imageDims = features.shape[1:3]
generator = getUnet(imageDims)
generator.summary()

To check that the generator model is working, we initialise an untrained generator and pass through some example data.

In [ ]:
# Load data and normalize
newFeatures, newTargets = passDataThroughSigmoid(features, targets, minMaxes)
newFeatures, newTargets = features*2 - 1, targets*2 - 1

# Initialise a generator model
imageDims = newFeatures.shape[1:3]
generator = getUnet(imageDims)

# Pass through some example data and viusualize the untrained model
batchSize = 2
featuresBatch = newFeatures[0:batchSize]
noise = np.random.normal(0, 1, (batchSize, 32, 16, 1024))
generatedImageBatch = generator([featuresBatch, noise], training=False).numpy()
print('Min: {}, Max: {}'.format(np.min(generatedImageBatch), np.max(generatedImageBatch)))
fig, axis = visualizeAsImages(featuresBatch[0], generatedImageBatch[0])
axis[2].set_title('Generated Output')

# Discriminator Model

The discriminator model attempts to distinguish *real* input produced by GOSPL from *fake* inputs produced by the generator model discussed above. It will take two images as inputs, one which is the same input given to the generator model, and one that is either generated by GOSPL (real) or by our generator model (fake). It's output will be a 2D array of predictions ranging from 0 to 1, based on how confident it is that each patch of the image is real or fake. Again, the code bellow has been taken and modified from *Eric Guerin et. al.*'s implementation. The structure of the discriminator is a series of downsampling convolutional layers.

In the combined model, the performance of the discriminator will be measured by a loss function, which uses binary cross entropy to measure how similar the discriminator output is to the ground truth. During training, the weights of the discriminator model will be adjusted to minimize this loss function.

In [ ]:
def getDiscriminatorModel(imageDims):
    imageWidth, imageHeight = imageDims[1], imageDims[0]
    
    #Create inputs of initial and generated image to discrimate, and combine them
    init = RandomNormal(stddev=0.02)
    initialImage = layers.Input(shape=(imageHeight, imageWidth, 2))
    generatedImage = layers.Input((imageHeight, imageWidth, 3))
    combinedImages = layers.Concatenate()([initialImage, generatedImage])
    
    #Main structure of the discriminator neural network model
    d = layers.Conv2D(64/4, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(combinedImages)
    d = layers.LeakyReLU(alpha=0.2)(d)
    d = layers.Conv2D(128/4, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d)
    d = layers.LeakyReLU(alpha=0.2)(d)
    d = layers.Conv2D(256/4, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d)
    d = layers.LeakyReLU(alpha=0.2)(d)
    d = layers.Conv2D(512/2, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d)
    d = layers.LeakyReLU(alpha=0.2)(d)
    d = layers.Conv2D(512/2, (4, 4), padding='same', kernel_initializer=init)(d)
    x = layers.LeakyReLU(alpha=0.2)(d)
    output = layers.Conv2D(1, (4, 4), padding='same', activation='sigmoid', kernel_initializer=init)(d)
    
    #Create the model and return it
    model = Model([initialImage, generatedImage], output)
    return model

# Create a discriminator model and print a summary of it's layers
discriminator = getDiscriminatorModel(imageDims)
discriminator.summary()

Pass data through to discriminator to confirm it is indeed working.

In [ ]:
decision = discriminator((featuresBatch, generatedImageBatch))
print(decision.shape)

# The Combined Model

Using the generator and discriminator models above, we can combine the two into a single tensorflow model.

In [ ]:
def combinedModel(generator, discriminator, imageDims):
    discriminator.trainable = False
    generatorInput = layers.Input(shape=(imageDims[0], imageDims[1], 2))
    inputNoise = layers.Input(shape=(32, 16, 1024))
    generatorOutput = generator([generatorInput, inputNoise])
    discriminatorOutput = discriminator([generatorInput, generatorOutput])
    model = Model(inputs=[generatorInput, inputNoise], outputs=[discriminatorOutput, generatorOutput])
    return model

#Create a mountDiscriminator and print summary of model
mountDiscriminator = combinedModel(generator, discriminator, imageDims)
mountDiscriminator.summary()

# Training Loop

Bellow are a few functions that will help with training our models.

In [ ]:
# Load data and normalize it using sigmoids
def loadAndPrepareData(dataSetDir='./Data/TrainingDataSets/Combined.npz'):
    featuresOriginal, targetsOriginal = loadDataset(dataSetDir=dataSetDir)
    features, targets = passDataThroughSigmoid(featuresOriginal, targetsOriginal)
    return features*2 - 1, targets*2 - 1

#Get number of patches in X and Y directions of the discriminator output
def getNXYpatches(features, batchSize, generator, discriminator):
    featuresBatch = features[0:batchSize]
    w_noise = np.random.normal(0, 1, (batchSize, 32, 16, 1024))
    generatedImageBatch = generator([featuresBatch, w_noise], training=False)
    decision = discriminator((featuresBatch, generatedImageBatch))
    nXPatches, nYPatches = decision.shape[1], decision.shape[2]
    return nXPatches, nYPatches

#Using the test feature image, we save generator output images to view the progress of it's training
def saveProgressImage(generator, testImage, iteration, dirName='./ProgressImages/Progress{}.png'):
    w_noise = np.random.normal(0, 1, (1, 32, 16, 1024))
    generatedImage = generator([testImage[np.newaxis], w_noise], training=False)[0]
    generatedImage = generatedImage.numpy()
    generatedImage = ((generatedImage + 1) / 2) * 255
    img = Image.fromarray(generatedImage.astype(np.uint8))
    img.save(dirName.format(iteration))

'''
#Using the test feature image, we save generator output images to view the progress of it's training
def saveProgressImage(generator, testImage, iteration, dirName='./ProgressImages/Progress{}.png'):
    w_noise = np.random.normal(0, 1, (1, 32, 16, 1024))
    generatedImage = generator([testImage[np.newaxis], w_noise], training=False)[0]
    generatedImage = generatedImage.numpy()
    generatedImage -= np.min(generatedImage)
    generatedImage /= np.max(generatedImage)
    generatedImage *= 256
    img = Image.fromarray(generatedImage.astype(np.uint8))
    img.save(dirName.format(iteration))
'''

We create a class named GAN. It will load and prepare our data, set up our models and write relevant outputs to file in the appropriate output directories.

It will initialise untrained generator and discriminator model, and compile them by specifying appropriate loss functions and optimization methods. A function is defined for creating real examples from the GOSPL training data, and another function is used to generate fake examples using the generator model. The discriminator will attempt to distinguish the two examples, and it's weights will be adjusted to improve at this task. The weights of the generator model will then be adjusted to improve it's ability to fool the discriminator.

The function *saveModels* will write the weights of the model to file, which can be used to load the model later on.

In [ ]:
class GAN:
    
    # Initialise GAN with various optional parameters
    def __init__(self,
                 batchSize = 4,
                 avg_loss = 0,
                 avg_dloss = 0,
                 
                 dataSetDir = './Data/TrainingDataSets/Combined.npz',
                 modelsMainDir = './SavedTensorflowModels', 
                 trialDirFormat = '{}/Model{}'
                ):
        
        # Store parameters as class properties
        self.batchSize = batchSize
        self.avg_loss = avg_loss
        self.avg_dloss = avg_dloss
        self.dataSetDir = dataSetDir
        self.modelsMainDir = modelsMainDir
        self.trialDirFormat = trialDirFormat
        
        
        self.features, self.targets = loadAndPrepareData(dataSetDir=dataSetDir)
        self.setUpModels(self.features.shape[1:3])
        self.nXPatches, self.nYPatches = getNXYpatches(self.features, self.batchSize, self.generator, self.discriminator)
        self.createModelsDir()
    
    # Do a single training step on all models
    def doTrainingStep(self, iteration):
    
        # Create real examples from data set and fake examples from generator model
        feats, realLabels, targs = self.generateRealSamples()
        fakeOutput, fakeLabels = self.generateFakeSamples(feats)

        # Training steps for composite and discriminator models
        w_noise = np.random.normal(0, 1, (self.batchSize, 32, 16, 1024))
        losses_composite = self.compositeModel.train_on_batch([feats, w_noise], [realLabels, targs])
        loss_discriminator_fake = self.discriminator.train_on_batch([feats, fakeOutput], fakeLabels)
        loss_discriminator_real = self.discriminator.train_on_batch([feats, targs], realLabels)
        saveProgressImage(self.generator, self.features[0], iteration, dirName=self.progressImageDir+'/Progress{}.png')

        # Print average loss so far
        d_loss = (loss_discriminator_fake + loss_discriminator_real) / 2
        self.avg_dloss += (d_loss - self.avg_dloss) / (i + 1)
        self.avg_loss += (losses_composite[0] - self.avg_loss) / (i + 1)
        print('total loss:' + str(self.avg_loss) + ' d_loss:' + str(self.avg_dloss))
        print('Dloss: {}, GenLoss: {}'.format(d_loss, losses_composite[0]))
    
    # Set up models with loss functions and optimizers
    def setUpModels(self, imageDims):
        self.generator = getUnet(imageDims)
        self.discriminator = getDiscriminatorModel(imageDims)
        adamOptimizer = tf.keras.optimizers.Adam(1e-4)
        self.discriminator.compile(loss='binary_crossentropy', optimizer=adamOptimizer)
        self.compositeModel = combinedModel(self.generator, self.discriminator, imageDims)
        self.compositeModel.compile(loss=['binary_crossentropy', 'mae'], loss_weights=[1, 3], optimizer=adamOptimizer)
    
    # Randomly select a portion of feature data along with their corresponding target data
    def generateRealSamples(self):
        idx = np.random.randint(0, self.features.shape[0], self.batchSize)
        feats = self.features[idx]
        targs = self.targets[idx]
        desiredDiscriminatorProbs = np.ones((self.batchSize, self.nXPatches, self.nYPatches, 1))
        return feats, desiredDiscriminatorProbs, targs

    # Generate fake images corresponding to the real targets
    def generateFakeSamples(self, featureBatch):
        w_noise = np.random.normal(0, 1, (featureBatch.shape[0], 32, 16, 1024))
        fakeOutput = self.generator.predict([featureBatch, w_noise])
        desiredDescriminatorProbs = np.zeros((len(fakeOutput), self.nXPatches, self.nYPatches, 1))
        return fakeOutput, desiredDescriminatorProbs
    
    # Within the main tensorflow models directory, create and return a subdirectory for this particular model
    def createModelsDir(self):
        self.trialNumber = 0
        self.trialDir = self.trialDirFormat.format(self.modelsMainDir, self.trialNumber)
        while os.path.isdir(self.trialDir):
            self.trialNumber += 1
            self.trialDir = self.trialDirFormat.format(self.modelsMainDir, self.trialNumber)
        self.progressImageDir = self.trialDir + '/ProgressImages'
        os.mkdir(self.trialDir)
        os.mkdir(self.progressImageDir)
    
    # Save the tensorflow model to file
    def saveModels(self, iteration):
        modelsSaveDir = self.trialDir + '/ModelsAtIteration{}'.format(iteration)
        os.mkdir(modelsSaveDir)
        self.generator.save(modelsSaveDir + '/generator.h5')
        self.discriminator.save(modelsSaveDir + '/discriminator.h5')

The code bellow will run the training of the GAN. It will create a new directory to save files to, and generate a progress image at each training iteration. Every N steps, it will also save the models weights to file.

Note that as of now, the resulting model varies significantly in quality. This is due to local minima in the loss functions that the weights get stuck in, which prevents us from finding the global minium. During training, make sure to monitor the output progress images saved to file, to ensure that the model is indeed improving in quality.

To stop the model from training, simply interupt the kernel.

In [ ]:
i = 0
trainThisGan = False
saveModelEveryNsteps = 200

if trainThisGan:
    #gan = GAN(dataSetDir='./Data/TrainingDataSets/Combined2.npz')
    gan = GAN(dataSetDir='./Data/TrainingDataSets/gosplData.npz')
    while True:
        i += 1
        gan.doTrainingStep(i)
        if i % saveModelEveryNsteps == 0:
            gan.saveModels(i)

# Load and Run a Saved Tensorflow Model

After training and saving various tensorflow models, we can load them using the code bellow. We are only saving the generator and discriminator components of the model, and not the overall combined model. These discarded components include loss functions, and optimization methods, which are only really required for training. If we would like to continue training a saved model, we will need to recompile them with the appropriate components.

In [ ]:
# Create a new GAN instance just like usual
gan = GAN(dataSetDir='./Data/TrainingDataSets/gosplData.npz')

# Load the weights to the generator and discriminator models
generatorModelDir = './SavedTensorflowModels/Model0/ModelsAtIteration13200/generator.h5'
discriminatorModelDir = './SavedTensorflowModels/Model0/ModelsAtIteration13200/discriminator.h5'
gan.generator.load_weights(generatorModelDir)
gan.discriminator.load_weights(discriminatorModelDir)

After loading the above model, take a random sample from the training data set and it's input, and use it to generate an output that we can visualize.

In [ ]:
batchSize = 2
rand = np.random.randint(0, gan.features.shape[0] - batchSize)
featuresBatch = gan.features[rand:rand+batchSize]
noise = np.random.normal(0, 1, (batchSize, 32, 16, 1024))
generatedImageBatch = gan.generator([featuresBatch, noise], training=False).numpy()
fig, axis = visualizeAsImages(featuresBatch[0], generatedImageBatch[0])

We would now like to see how the results look like on a spherical mesh, the format that we actually care about.

In [ ]:
# Pass a random input through the tensorflow model
batchSize = 2
rand = np.random.randint(0, gan.features.shape[0] - batchSize)
featuresBatch = gan.features[rand:rand+batchSize]
noise = np.random.normal(0, 1, (batchSize, 32, 16, 1024))
generatedImageBatch = gan.generator([featuresBatch, noise], training=False).numpy()

# Bring data from [-1, 1] to [0, 1]
featureSample = (featuresBatch[0]+1)/2
generatedImage = (generatedImageBatch[0]+1)/2

# Add some padding to the data range [0, 1] (to avoid infinities)
offset = 1e-7
featuresOffseted = (featureSample + offset) / (1+2*offset)
targetOffseted = (generatedImage + offset) / (1+2*offset)
resultFeatures, resultsTarget = passDataThroughLogit(featuresOffseted, targetOffseted)

# Visualise results as a mesh
plotter = pv.PlotterITK()
resultsMesh = createExageratedMesh(resultFeatures, resultsTarget, amplificationFactor=120, earthRadius=6378137)
plotter.add_mesh(resultsMesh, scalars="Final Elevations")
plotter.show()

From the above results, we can see that our model is capable of producing an output that somewhat resembles GOSPL simulation, but some clear defects are visible. The river networks are not that visible, and instead we have a weirdly rough topography.

**How to Improve:** A potential major source of error is the fact that our discriminator is working with data within the sigmoid transformed space used in preprocessing, and not within the original data domain. This means our generator model is learning how to generate fake examples within the sigmoid domain. To fix this, the discriminator needs to compare sample within the original data domain, and not the transformed space.

# Testing Model on Unseen Input Data

To trully test how well the model perform, we need to pass it some example input data that was not included in the training data. This is to make sure that the model generalizes well to unseen data, and avoid overfitting. We begin by redefining the noise functions used to generate the original input data. After randomly generating new input data, we pass it through the generator model, and visualise it as a spherical mesh.

Our generator model seems to perform just as well on unseed inputs as it does with inputs included in the trianing stage, which means we do not have a problem of overfitting. To further test the capabilities of our model, we can change the noise parameters used to generate the initial topography and uplit. Doing this will test the model on noise frequencies that were not at all included in the training data.

The model works surprising well on uplift generated by low frequency noise (change *octaveStepSize* to 0.8 within uplift noise parameters).

In [ ]:
#Sample noise with multiple noise frequencies
def sampleNoise(XYZ, noiseParameters):
    noiseSum = np.zeros(XYZ.shape[0])
    for i in range(noiseParameters["octaves"]):
        frequency =  noiseParameters["initialFrequency"] * noiseParameters["octaveStepSize"] ** i
        noiseSum += samplePerlinNoise(XYZ, frequency=frequency) / (noiseParameters["amplitudeStepSize"]*(i+1))

    # Normalise noise and bring to desired noise range
    minMax = noiseParameters["noiseMinMax"]
    noiseSum -= np.min(noiseSum)
    noiseSum = noiseSum * (minMax[1] - minMax[0]) / np.max(noiseSum) + minMax[0]
    return noiseSum

# Given a list of XYZ coordinates, we sample a noise value at each point
def samplePerlinNoise(XYZ, amplitude=1, frequency=4, offset=None):
    if offset == None: #Random offset if none other is specified
        offset = np.random.rand(3) * 100000
    freq = (frequency, frequency, frequency)
    noise = pv.perlin_noise(amplitude, freq, offset)
    return np.array([noise.EvaluateFunction(xyz) for xyz in XYZ])


# Parameters used to generate original input data
earthRadius = 6378137
sphereResolution = [512, 258]
topographyNoiseParameters = {
    "octaves" : 8,
    "octaveStepSize" : 1.4,
    "amplitudeStepSize" : 2.0,
    "initialFrequency" : 0.000001,
    "noiseMinMax" : np.array([-4000, 1000])}
upliftNoiseParameters = {
    "octaves" : 6,
    "octaveStepSize" : 1.4, # Change this to significantly change noise frequencies of uplift
    "amplitudeStepSize" : 2.4,
    "initialFrequency" : 0.0000002,
    "noiseMinMax" : np.array([0, 2000])}

# Create a sphere and generate noise based inputs
start = time.time()
sphere = pv.Sphere(theta_resolution=sphereResolution[0], phi_resolution=sphereResolution[1], radius=earthRadius)
topography = sampleNoise(sphere.points, topographyNoiseParameters)
uplift = sampleNoise(sphere.points, upliftNoiseParameters)
print('Took {} seconds to generate input data'.format(time.time() - start))
start = time.time()

# Preprocess initial topography input
topoNormalized = topography.T[2:]
topoNormalized = topoNormalized.reshape(512, 256)
topoNormalized = sigmoid(topoNormalized, minMax=[-4000, 1000])
topoNormalized = 2 * topoNormalized - 1

# Preprocess tectonic uplift input
upliftNormalized = uplift.T[2:] / 10000000
upliftNormalized = upliftNormalized.reshape(512, 256)
upliftNormalized = sigmoid(upliftNormalized, minMax=[0, 0.0002])
upliftNormalized = 2 * upliftNormalized - 1

# Pass randomly generated input data through the tensorflow model
noise = np.random.normal(0, 1, (1, 32, 16, 1024))
inputBatch = np.array([np.stack((topoNormalized, upliftNormalized), axis=-1)])
print('Took {} seconds to preprocess data'.format(time.time() - start))
start = time.time()

generatedImageBatch = gan.generator([inputBatch, noise], training=False).numpy()
print('Took {} seconds pass data through sigmoid'.format(time.time() - start))
start = time.time()

# Bring output data back to original data domain
offset = 1e-2
featureSample = (inputBatch[0]+1)/2
generatedImage = (generatedImageBatch[0]+1)/2
featuresOffseted = (featureSample + offset) / (1+2*offset)
targetOffseted = (generatedImage + offset) / (1+2*offset)
resultFeatures, resultsTarget = passDataThroughLogit(featuresOffseted, targetOffseted)
print('Took {} seconds to postprocess data'.format(time.time() - start))

# Plot the results
plotter = pv.PlotterITK()
resultsMesh = createExageratedMesh(resultFeatures, resultsTarget, amplificationFactor=120, earthRadius=6378137)
plotter.add_mesh(resultsMesh, scalars="Final Elevations")
plotter.show()

We can compare the above output with a real example generated by GOSPL.

In [ ]:
# Create exagerated mesh
sampleNumber = 10
features, targets = loadDataset('./Data/TrainingDataSets/SmallExampleDataset.npz')
exageratedMesh = createExageratedMesh(features[sampleNumber], targets[sampleNumber])

# Plot the results
plotter = pv.PlotterITK()
plotter.add_mesh(exageratedMesh, scalars='Final Elevations')
plotter.show()